In [ ]:
#Script to open netcdf and plot
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cartopy #Instead of from mpl_toolkits.basemap import Basemap "conda install -c conda-forge cartopy"
from netCDF4 import Dataset #"conda install -c anaconda netcdf4"
import cartopy.crs as ccrs
import cartopy.feature as cfeature


path = '.nc'#Path to ERA5 wind file
outpath = ''#outpath for jpgs

#Wind speed sqrt(u^2 + v^2) - see ERA5 doca


from mpl_toolkits.axes_grid1 import make_axes_locatable

data = Dataset(path)

In [ ]:
#Hourly to daily

import xarray as xr                                                    # import xarray library. Necessary for daily

lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
#wind = data.variables['si10'][:,:,:] #wind
#u = data.variables['u10'][:,:,:]
#v = data.variables['v10'][:,:,:]
#Extra steps to get daily from hourly data
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]

u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 

windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)

In [ ]:
#Plot as daily jpgs

days = np.arange(0,366) #30 days (Mar-Dec). No of days
#note 2020 is a leap year. Mar1 is day 61

for i in days:

    
    plt.figure(figsize=(10,10), dpi= 90)
    ax = plt.axes(projection=ccrs.SouthPolarStereo(true_scale_latitude=-70))
    theplot = plt.pcolormesh(lons,lats,windspeed[i,:,:],transform=ccrs.PlateCarree(),cmap = 'jet', vmin=0, vmax=14) #Set colorbar max/min
    plt.quiver(lons, lats, u[i,:,:], v[i,:,:], pivot='middle', transform=ccrs.PlateCarree())

    ax.coastlines()
    ax.set_extent([-120, -106, -74.6, -72], ccrs.PlateCarree())
    
    DoY = i+1
    plt.title('Wind Speed (m/s), 2020 Day of year: ' + str(DoY)) #Edit this number based on when data stars in year

    cbar = plt.colorbar(theplot, shrink=0.75)

    plt.savefig(outpath+str(DoY)+'.jpg')
    

In [ ]:
#Script to save collection of jpgs to a GIF. And a script to then convert that GIF to an mp4 if necessary.
#Thank you to https://www.youtube.com/watch?v=dSv3-obKv3M
from PIL import Image
import numpy as np

image_frames = []
months = np.arange(1,367) 

jpegspath = outpath #DEFINE Path to folder -- same as outpath above here
for k in months:
    new_frame = Image.open(jpegspath + str(k) + '.jpg') #Here the images are 3-(March-) so need to add 2
    image_frames.append(new_frame) #Append each image

gifname = outpath+'.gif' #DEFINE name of gif to save. 
image_frames[0].save(gifname,format = 'gif', #
                    append_images = image_frames[1:], #1 means second onwards
                    save_all = True, duration = 250)#units is milliscenonds for each frame
                     #1 means second onwards  loop = 0

#Now make it a movie
import moviepy.editor as mp #pip install MoviePy

gifpath = outpath+'.gif' #DEFINE Path to where gif is stored
clip = mp.VideoFileClip(gifpath) 
clip.write_videofile(outpath+".mp4") #DEFINE movie OUTPUT NAME and path
